# Semantische Suche

Erstellt Embeddings (Vektoren), die die Bedeutung der gegebenen Inhalte in 200 Dimensionen kodiert. Für die Umwandlung von Text bzw. Wörtern in sog. Embeddings wurde ein Model trainiert, auf das hier zurückgegriffen wird.

Folgendes Beispiel zeigt, wie man mit diesen Vektoren rechnen kann. Subtrahiert man woman vom Wort mother und addiert man man erhält man father, wie du im folgenden Beispiel siehst. (Hinweis: Der Download kann etwas länger dauern).

Hast du noch weitere Ideen, mit welchen Wörtern man rechnen kann?

z.B. teacher - school + university = professor
z.B. pretzel - germany + france ≈ baguette


In [1]:
%pip install -q gensim
import gensim.downloader as api
model = api.load("glove-wiki-gigaword-200")




[notice] A new release of pip is available: 23.1.2 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
[==================================================] 100.0% 252.1/252.1MB downloaded


In [2]:
vec = model["mother"] - model["woman"] + model["man"]


model.similar_by_vector(vec, topn=5)

[('father', 0.8118535280227661),
 ('mother', 0.7553707957267761),
 ('man', 0.7454989552497864),
 ('brother', 0.7374289035797119),
 ('son', 0.7233688235282898)]

Im nächsten Beispiel nutzen wir diesen Technologie, um die Bedeutung von Texten zu durchsuchen. Dafür nutzen wir eine Liste an Texten und ein Suchbegriff, die von einem Modell in Vektoren umgewandelt werden. Mithilfe des Skalarprodukts kann dann über alle Dimensionen die Ähnlichkeit von zwei Vektoren ermittelt werden. Dabei ist der Wert größer je "mehr" die Vektoren in ihrer Richtung übereinstimmen. Dabei ist es einfacher sich das ganze im 3-Dimensionalen Raum vorzustellen.

Für diese Aufgabe ist allerdings ein anderes Model nötig, da dieses ganze Sätze verstehen kann. Dies konvertiert die Sätze in 768 dimensionale Vektoren.

In [3]:
%pip install -q sentence_transformers
%pip install -q torch tf-keras
from sentence_transformers import SentenceTransformer, util
import torch

model = SentenceTransformer("sentence-transformers/paraphrase-mpnet-base-v2")


[notice] A new release of pip is available: 23.1.2 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.1.2 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


/home/vscode/.local/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-05-19 09:55:38.663058: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747648538.677999    4397 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747648538.682476    4397 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1747648538.695647    4397 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00

In [4]:

sentences = [
    "Deep learning models are revolutionizing natural language processing.",
    "Reinforcement learning algorithms are used in robotics and game playing.",
    "Generative adversarial networks are creating realistic images and videos.",
    "Explainable AI is crucial for building trust in AI systems.",
    "Ethical considerations in AI development are becoming increasingly important."
]

len(sentences)

5

In [5]:
encodings = model.encode(sentences)

# Das ist die Frage, die mit den gebenen Sätze verglichen wird?
# Welcher Satz passt den deiner Meinung nach zu folgender Frage?
query = "Which model is used for media creation?"

queryEncoding = model.encode(query)

print(queryEncoding)

# Mithilfe der Softmax Funktion werden die Ähnlichkeiten normiert und als Wahrscheinlichkeitsverteilung ausgegeben
result = torch.nn.Softmax(1)(util.cos_sim(queryEncoding, encodings),)
result


[ 2.17507686e-02 -1.58684388e-01 -8.73355474e-03 -7.93801025e-02
  6.58603758e-02  4.32269052e-02  3.18210930e-01 -6.65029734e-02
 -2.10551068e-01 -1.46435559e-01  4.53497544e-02  4.06425148e-02
 -2.09189318e-02  2.17503548e-01  1.11100808e-01 -1.48284182e-01
  8.40316042e-02  1.30630851e-01 -1.76880702e-01  7.36415982e-02
  2.94402416e-04 -4.31842655e-02 -4.41200193e-03  1.00005958e-02
 -3.15263569e-01 -2.16687873e-01 -1.94415554e-01  5.87974489e-02
 -4.42587724e-03  7.49052390e-02 -6.34334097e-03  1.98147580e-01
  1.56810239e-01  1.27115369e-01  3.20420414e-02 -9.47865769e-02
 -4.72161621e-02 -7.15053156e-02 -3.95139083e-02  2.39444539e-01
  4.23232257e-01  1.69619411e-01 -9.28514525e-02 -4.95444052e-02
 -8.01617354e-02  1.64747387e-02  2.96002328e-01 -1.96525395e-01
  8.50908905e-02  9.59708020e-02 -6.30307272e-02 -4.90415171e-02
  1.38531551e-01  6.90234676e-02 -1.19049631e-01  1.51550815e-01
 -9.94316265e-02  2.30667681e-01  2.85095751e-01  1.23887338e-01
  1.13073789e-01  6.53659

tensor([[0.1857, 0.1904, 0.2388, 0.1931, 0.1920]])

In [6]:
# Der höchste Wert wird gesucht und der beste passendste Satz wird ausgegeben

# Die Argmax-Funktion gibt den Index des höchsten Werts zurück
highest = torch.argmax(result)
print(highest)
print(sentences[highest])
#

tensor(2)
Generative adversarial networks are creating realistic images and videos.


- Aufgabe: Finde drei Wortpaare, die sich semantisch ähnlich sind (z.B. glücklich - zufrieden).
Schwierigkeitsgrad: leicht

- Aufgabe: Erkläre mit eigenen Worten, was ein Embedding ist.
Schwierigkeitsgrad: leicht

- Aufgabe: Verändere den Suchbegriff (query) im Code und beobachte, wie sich das Ergebnis ändert. Beschreibe deine Beobachtungen.
Schwierigkeitsgrad: mittel

- Aufgabe: Füge drei weitere Sätze zur Liste 'sentences' hinzu und überprüfe, wie das Ergebnis der semantischen Suche beeinflusst wird.
Schwierigkeitsgrad: mittel

- Aufgabe: Recherchiere, welche anderen SentenceTransformer-Modelle es gibt und vergleiche deren Leistung mit dem gegebenen Modell.
Schwierigkeitsgrad: mittel

- Aufgabe: Implementiere eine eigene Funktion, die die Ähnlichkeit zwischen zwei Sätzen berechnet, ohne die 'util.cos_sim' Funktion zu verwenden.
Schwierigkeitsgrad: schwer

- Aufgabe: Erweitere den Code so, dass die Top 3 ähnlichsten Sätze ausgegeben werden, statt nur des ähnlichsten.
Schwierigkeitsgrad: mittel